# broken-machine challange - quite challangeable

References:

https://www.geeksforgeeks.org/using-learning-curves-ml/

https://datascience.stackexchange.com/questions/26918/validation-curve-unlike-sklearn-sample

https://gist.github.com/otaviomguerra/51df7a4cff28f92de7105f12a0724115

https://datatofish.com/k-means-clustering-python/

https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray

https://www.kaggle.com/ellecf/visualizing-multidimensional-clusters

https://community.greatlearning.in/t/while-using-pairplot-using-seaborn-package-i-am-getting-error-as-selected-kde-bandwidth-is-0-cannot-estiamte-density-can-someone-please-help-on-this/158/7

https://www.geeksforgeeks.org/python-seaborn-pairgrid-method/

# Broken Machine dataset is used with following main steps:
* Fill missing values with mode
* Find correlation between features
* Undersample data set as we've got 900,000 rows of data with almost 70%-30% distribution of labels
* Use scaling (StandardScaler)
* Do ramdomizedSearchCV to select initialized parameters
* Plot learning curves over multiple iterations
* Plot validation curves over multiple iterations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import validation_curve
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
# !pip install pydotplus
# import pydotplus
from IPython.display import Image
from sklearn.model_selection import learning_curve 
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, precision_score, recall_score, roc_curve, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from joblib import dump, load
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
# !pip install --upgrade pip
# !pip list | grep -i imb
# !pip uninstall -y imbalanced-learn 
# !pip list | grep -i imb
!pip install imblearn
# !pip list | grep -i imb
# !pip uninstall -y imbalanced-learn 
# !pip list | grep -i imb
from imblearn.under_sampling import NearMiss
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import sklearn.metrics as metrics
from sklearn.metrics import homogeneity_score

In [ ]:
file_path = '../input/the-broken-machine/'
model_path = '../input/the-broken-machine/'
# file_path = './the-broken-machine/'

In [ ]:
xtrain = pd.read_csv(file_path + 'xtrain.csv')
ytrain = pd.read_csv(file_path + 'ytrain.csv')
print(xtrain.shape)
print(ytrain.shape)
xtrain.head()

In [ ]:
ytrain.head()

In [ ]:
ytrain.info()

In [ ]:
xtrain.info()

In [ ]:
print("1 ratio is：",ytrain[ytrain==1].count()/len(ytrain)*100)
#Then the accuracy is less than 70% is meaningless

In [ ]:
pd.value_counts(ytrain.values.flatten())

In [ ]:
# y train percentage 1 %
pd.value_counts(ytrain.values.flatten())[1]/(pd.value_counts(ytrain.values.flatten())[0]+pd.value_counts(ytrain.values.flatten())[1])*100

In [ ]:
#check data
pd.set_option('display.max_columns', None)
xtrain.describe()

In [ ]:
#Check missing data
all_data_na = (xtrain.isnull().sum() / len(xtrain)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
f, ax = plt.subplots(figsize=(8, 6))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
%%time
#EDA NA processing,lgb doesn't need na processing
for col in xtrain.columns:
    xtrain[col] = xtrain[col].fillna(xtrain[col].mode()[0])#mode
xtrain.describe()

In [ ]:
xtrain.isnull().sum()

In [ ]:
xtrain.head()

In [ ]:
# EDA skew
xtrain.skew(axis=0).sort_values(ascending=False)
#Found 37 numerical anomalies

In [ ]:
xtrain['37'].hist()

In [ ]:
xtrain['37']=xtrain['37'].apply(lambda x:200 if x>100 else x) #Handling No. 37

In [ ]:
#EDA No. 37

def check_skewness(col):
    sns.distplot(xtrain[col] , fit=norm);
    fig = plt.figure()
#     res = stats.probplot(xtrain[col], plot=plt) #Probplot cannot be displayed, if it is an integer index, it can be displayed
    # Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(xtrain[col])
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
    
check_skewness(['37']) 

In [ ]:
# check unique value
for i in xtrain.columns:
    print(i,": ",len(xtrain[i].unique()))

In [ ]:
#Feature distribution

h = .2  # step size in the mesh

x_min, x_max = xtrain.iloc[0:1000, 33].min() - .5, xtrain.iloc[0:1000, 33].max() + .5
y_min, y_max = xtrain.iloc[0:1000, 36].min() - .5, xtrain.iloc[0:1000, 36].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
# just plot the dataset first
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
ax = plt.subplot()
ax.scatter(xtrain.iloc[0:1000, 33], xtrain.iloc[0:1000, 36], c=list(ytrain.iloc[0:1000,0]),cmap=cm_bright,
           edgecolors='k')
ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.set_xticks(())
ax.set_yticks(())

In [ ]:
xtrain['1'].hist()

In [ ]:
#corelation
corrmat = xtrain.corr()
corrmat

In [ ]:
corrmat[corrmat>0.01].count()
#No clear corelation

In [ ]:
# plt.figure(figsize=(10,10))
# g = sns.heatmap(train_data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.check_cv.html#sklearn.model_selection.check_cv
# from sklearn.model_selection import check_cv
# cv = check_cv(3, xtrain, ytrain, classifier=True)

No clear patten for the scattering

In [ ]:
xy = xtrain.join(ytrain)
train_sample = xy.sample(n=17000, random_state=0)
pd.value_counts(train_sample['x'].values.flatten())
X = train_sample.iloc[:, :-1]
y = train_sample.iloc[:,-1]
!pip install imblearn -U
from imblearn.under_sampling import NearMiss
ns=NearMiss()
X_train_ns,y_train_ns=ns.fit_resample(X,y)
X_train_ns.shape

In [ ]:
pd.value_counts(y_train_ns.values.flatten())

## Training part

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
X_balanced, X_test_balanced, y_balanced, y_test_balanced=train_test_split(X_train_ns[0:-1000],y_train_ns[0:-1000], test_size=0.2, random_state=3)
# gc.collect()  
print(X_balanced.shape)
print(X_test_balanced.shape)
print(y_balanced.shape)
print(y_test_balanced.shape)

In [ ]:
%%time

ss = StandardScaler()
X_balanced = pd.DataFrame(ss.fit_transform(X_balanced), columns=X_balanced.columns)
X_test_balanced = pd.DataFrame(ss.transform(X_test_balanced), columns=X_test_balanced.columns)
# we have now fit and transform the data into a scaler for accurate reading and results.

In [ ]:
X_balanced.head()

In [ ]:
X_balanced

In [ ]:
type(X_balanced)

In [ ]:
type(y_balanced.to_frame())

In [ ]:
y_balanced.to_frame()

# Algos code start here

KMeans


In [ ]:
sScore = []
hScore = []
k = range(2,50)
# k = range(2,4)
for i in k:
    labels=KMeans(n_clusters=i,random_state=3).fit(X_balanced).labels_
    sScore.append(metrics.silhouette_score(X_balanced,labels,random_state=3))
#     print ("Silhouette score for k(clusters) = "+str(i)+" is "
#            +str(metrics.silhouette_score(X_balanced,labels,random_state=3)))
    hScore.append(homogeneity_score(y_balanced, labels))

plt.plot(k, sScore, 'bx-', color='red',label="Sillheaoute score")
plt.plot(k, hScore, 'bx-', color='blue',label="Homogeniety")
plt.xlabel('k')
# plt.ylabel('AIC/BIC')
plt.grid()
plt.legend()
plt.show() 

In [ ]:
sScore = []
hScore = []
k = range(2,10)
# k = range(2,3)
for i in k:
    labels=KMeans(n_clusters=i,random_state=3).fit(X_balanced).labels_
    sScore.append(metrics.silhouette_score(X_balanced,labels,random_state=3))
#     print ("Silhouette score for k(clusters) = "+str(i)+" is "
#            +str(metrics.silhouette_score(X_balanced,labels,random_state=3)))
    hScore.append(homogeneity_score(y_balanced, labels))

plt.plot(k, sScore, 'bx-', color='red',label="Sillheaoute score")
plt.plot(k, hScore, 'bx-', color='blue',label="Homogeniety")
plt.xlabel('k')
# plt.ylabel('AIC/BIC')
plt.grid()
plt.legend()
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=3).fit(X_balanced)
y_pred_kmeans = kmeans.predict(X_balanced)
# y_pred_kmeans = KMeans(n_clusters=2, random_state=3).fit_predict(X_balanced)



In [ ]:
y_balanced = y_balanced.to_frame()
y_balanced=y_balanced.squeeze()

In [ ]:
labels = pd.DataFrame(kmeans.labels_) #This is where the label output of the KMeans we just ran lives. Make it a dataframe so we can concatenate back to the original data
labeled_X_balanced = pd.concat((X_balanced,labels),axis=1)
labeled_X_balanced = labeled_X_balanced.rename({0:'labels'},axis=1)

In [ ]:
m=TSNE(learning_rate=50, random_state=3 )
tsne_features = m.fit_transform(X_balanced)
X_balanced['t1']=tsne_features[:,0]
X_balanced['t2']=tsne_features[:,1]


finalDf = pd.concat([X_balanced, labels], axis = 1)
finalDf = finalDf.rename({0:'labels'},axis=1)
sns.scatterplot(x="t1",y="t2",hue='labels', data=finalDf)
plt.show()

In [ ]:
finalDf = pd.concat([X_balanced, y_balanced], axis = 1)
# finalDf = finalDf.rename({0:'labels'},axis=1)
sns.scatterplot(x="t1",y="t2",hue='x', data=finalDf)
plt.show()

In [ ]:
homogeneity_score(y_balanced, y_pred_kmeans)